In [1]:
import numpy as np
import pandas as pd
from convokit import Corpus, download
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

In [2]:
corpus = Corpus(filename=download('supreme-corpus'))

Dataset already exists at /Users/amaribauer/.convokit/downloads/supreme-corpus


In [54]:
# All cases
cases = pd.read_json(path_or_buf='data/cases.jsonl', lines=True)

# Cases with clear winners
df = cases.loc[cases.loc[:, 'win_side'].isin([1, 0])]

# Roberts court cases with clear winners
roberts = df.loc[df.loc[:, 'court'] == 'Roberts Court', :]

# All utterances
all_utts = corpus.get_utterances_dataframe()

# Roberts court case utterances
roberts_ids = roberts.loc[:, 'id'].unique()
utts = all_utts.loc[all_utts.loc[:, 'meta.case_id'].isin(roberts_ids)]

# Roberts court cases with clearn winners and utterance data
    # Unique case IDs from utts indicate the number of Roberts court cases
    # the corpus has utterance data for
subset_ids = utts.loc[:, 'meta.case_id'].unique()
    # Use these ids to subset the roberts data frame
    # (exclude cases without utterance data)
roberts = roberts.loc[roberts.loc[:, 'id'].isin(subset_ids)]
print(len(roberts), 'cases')
petitioner_wins = roberts.loc[:, 'win_side'].mean()
print(round(petitioner_wins * 100, 2),
      'percent of cases were decided favorably for the petitioner')

1024 cases
65.33 percent of cases were decided favorably for the petitioner


In [55]:
# Utterances per case
print('Min:', utts.groupby(['meta.case_id']).size().min()) 
print('Max:', utts.groupby(['meta.case_id']).size().max())
print('Mean', utts.groupby(['meta.case_id']).size().mean())

Min: 1
Max: 1235
Mean 237.3955078125


In [63]:
# Drop single-utterance cases
utt_counts = pd.DataFrame(utts.groupby(['meta.case_id']).size())
utt_counts = utt_counts.reset_index()
utt_counts = utt_counts.rename(columns={0: 'utt_counts'})
utts = pd.merge(utts, utt_counts, how = 'left')
utts = utts.loc[utts.loc[:, 'utt_counts'] != 1, :]
print('1 case dropped')
print('New min:', utts.groupby(['meta.case_id']).size().min()) 
print(len(utts.groupby(['meta.case_id'])), 'cases')




1 case dropped
New min: 66
1023 cases


In [8]:
utts['text'].replace('\d+', '', regex=True, inplace = True) #remove numbers, decide as group if this makes sense
group_utts = utts.groupby('meta.case_id')['text'].apply(' '.join)
#NOTE: No stemming or lemmatization done at this point
df = pd.merge(group_utts,roberts[['id','win_side']], how = 'left', 
              left_on = 'meta.case_id', right_on = 'id')
X_train, X_test, y_train, y_test = train_test_split(
    df.loc[:,df.columns != 'win_side'],df['win_side'], test_size = 0.2, random_state = 0)

In [9]:
#Initialize Vectorizer and vectorize train data
count_vect = CountVectorizer(stop_words = 'english') #play around with ngrams, max/min df args
count_df = count_vect.fit_transform(X_train['text'])
count_array = count_df.toarray()
count_df = pd.DataFrame(count_array,columns = count_vect.get_feature_names(), index = X_train['id'])

In [10]:
#Vectorize test data
count_test = count_vect.transform(X_test['text'])
count_test_array = count_test.toarray()
count_test_df = pd.DataFrame(count_test_array, columns = count_vect.get_feature_names(),index = X_test['id'])

In [12]:
"""
COUNT_TEST_DF AND COUNT_DF ARE OUR CURRENT BAG OF WORDS FEATURE SETS. I WILL UPDATE THEM AFTER IMPLEMENTING TF-IDF
TO TAKE INTO ACCOUNT CASE LENGTH
"""
count_df.to_csv('data/train_unigram_BoW.csv')
count_test_df.to_csv('data/test_unigram_BoW.csv')